In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import argparse

plt.switch_backend('agg')
# In[3]:
font = {'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import transforms, utils, models, datasets
from torch.utils.data import Dataset, DataLoader

In [2]:

from synthetic_dataloader import *


from synthetic_utils import *


# In[5]:


from trainer_dataloader import *
from networks import *
from losses import *


# In[6]:

In [3]:

input_size = 96
batch_size = 64
num_workers = 4


cuda = False
pin_memory = False
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    cuda = True
    cudnn.benchmark = True
    pin_memory = True
else:
    device = torch.device("cpu")

print('Device set: {}'.format(device))


# In[8]:


data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]),
        'val': transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
}



Device set: cuda


In [4]:
# if independent:
DATA_PATH = '/home/s1791387/diss/gen_shapes/'
# else:
#     DATA_PATH = '/home/s1791387/diss/dependent_gen/'
print(DATA_PATH)
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
VAL_PATH = os.path.join(DATA_PATH, 'valid')
TEST_PATH = os.path.join(DATA_PATH, 'test')

/home/s1791387/diss/gen_shapes/


In [5]:
train_df = datasets.ImageFolder(root=TRAIN_PATH, transform=data_transforms['train'])
val_df = datasets.ImageFolder(root=VAL_PATH, transform=data_transforms['val'])
test_df = datasets.ImageFolder(root=TEST_PATH, transform=data_transforms['val'])

In [6]:
LOAD_PATH = './dp2_nosig_indep/1/weights/encoder_173_0.722.pth'

In [7]:
laftr_encoder = torch.load(LOAD_PATH)

In [8]:
gender_train = GenderDataset(train_df)
gender_valid = GenderDataset(val_df)

In [9]:
advtrain_loader = DataLoader(gender_train, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
advval_loader = DataLoader(gender_valid, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)

In [10]:
adversary = ClassNet()

adv_criterion = nn.BCELoss()
opt_adv = optim.Adam(adversary.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler_adv = lr_scheduler.StepLR(optimizer=opt_adv, gamma=0.99, step_size=1)

In [11]:
num_epochs = 20
train_losses = []
train_accs = []
val_losses = []
val_accs = []
epoch_time = AverageMeter()
ep_end = time.time()
for epoch in range(0, num_epochs):
        print('Epoch: {}/{}'.format(epoch, num_epochs))
        scheduler_adv.step()
        # train
        train_loss, train_acc = train_classifier_epoch(advtrain_loader, laftr_encoder,
                                adversary, opt_adv, adv_criterion, device)
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        # validate
        print('-'*10)
        val_loss, val_acc = validate_classifier_epoch(advval_loader, laftr_encoder, adversary,
                                 adv_criterion, device)

        print('Avg validation loss: {} \t Accuracy: {}'.format(val_loss, val_acc))
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        print('-' * 20)
        epoch_time.update(time.time() - ep_end)
        ep_end = time.time()
        print('Epoch {}/{}\t'
              'Time {epoch_time.val:.3f} sec ({epoch_time.avg:.3f} sec)'.format(epoch, num_epochs, epoch_time=epoch_time))
        print('-'*20)

Epoch: 0/20


/home/s1791387/miniconda3/envs/diss/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Batch: [0/79]	Time 6.970 (6.970)	Loss 0.6961 (0.6961)	Accuracy 0.4688 (0.4688)
Batch: [10/79]	Time 0.126 (1.578)	Loss 0.7134 (0.7295)	Accuracy 0.5312 (0.4673)
Batch: [20/79]	Time 4.796 (1.563)	Loss 0.7121 (0.7185)	Accuracy 0.4688 (0.4807)
Batch: [30/79]	Time 0.243 (1.423)	Loss 0.6877 (0.7114)	Accuracy 0.5312 (0.4869)
Batch: [40/79]	Time 4.358 (1.419)	Loss 0.7034 (0.7075)	Accuracy 0.5000 (0.4928)
Batch: [50/79]	Time 0.077 (1.355)	Loss 0.6821 (0.7045)	Accuracy 0.6250 (0.5015)
Batch: [60/79]	Time 4.411 (1.383)	Loss 0.6881 (0.7026)	Accuracy 0.5469 (0.5072)
Batch: [70/79]	Time 0.121 (1.368)	Loss 0.6888 (0.7009)	Accuracy 0.6094 (0.5114)
----------


/home/s1791387/miniconda3/envs/diss/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Avg validation loss: 0.7043784914016724 	 Accuracy: 0.504
--------------------
Epoch 0/20	Time 125.226 sec (125.226 sec)
--------------------
Epoch: 1/20


/home/s1791387/miniconda3/envs/diss/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([52])) that is different to the input size (torch.Size([52, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Batch: [0/79]	Time 9.767 (9.767)	Loss 0.6944 (0.6944)	Accuracy 0.4844 (0.4844)
Batch: [10/79]	Time 0.195 (2.071)	Loss 0.6972 (0.6937)	Accuracy 0.4844 (0.5142)
Batch: [20/79]	Time 6.278 (2.005)	Loss 0.6807 (0.6943)	Accuracy 0.5156 (0.5052)
Batch: [30/79]	Time 0.165 (1.781)	Loss 0.6928 (0.6935)	Accuracy 0.5312 (0.5111)
Batch: [40/79]	Time 6.127 (1.790)	Loss 0.6883 (0.6927)	Accuracy 0.5469 (0.5179)
Batch: [50/79]	Time 0.135 (1.686)	Loss 0.6914 (0.6931)	Accuracy 0.5625 (0.5165)
Batch: [60/79]	Time 6.324 (1.708)	Loss 0.7194 (0.6933)	Accuracy 0.4375 (0.5161)
Batch: [70/79]	Time 0.167 (1.641)	Loss 0.7026 (0.6934)	Accuracy 0.5000 (0.5145)
----------
Avg validation loss: 0.6885415554046631 	 Accuracy: 0.524
--------------------
Epoch 1/20	Time 144.451 sec (134.838 sec)
--------------------
Epoch: 2/20
Batch: [0/79]	Time 9.603 (9.603)	Loss 0.6947 (0.6947)	Accuracy 0.5000 (0.5000)
Batch: [10/79]	Time 0.235 (2.070)	Loss 0.6915 (0.6928)	Accuracy 0.4844 (0.5227)
Batch: [20/79]	Time 5.680 (1.994)	Los

No handles with labels found to put in legend.


In [12]:
plt.figure(figsize=(20,20))
plt.subplot(221)
plt.title('training classification loss')
plt.plot(train_losses)
plt.subplot(222)
plt.title('training accuracy')
plt.plot(train_accs)
plt.subplot(223)
plt.title('validation loss')
plt.plot(val_losses)
plt.subplot(224)
plt.title('validation accuracy')
plt.plot(val_accs)
plt.legend()
plt.tight_layout()